# Neuneier (1997) — Q-Learning multi‑activo con aversión al riesgo (utilidad)

Baseline educativo: portafolio con **N activos** + cash, política discreta de pesos por *softmax*, utilidad log para aversión al riesgo, costos de rebalanceo.


**Contenido:**
1) Serie sintética multiactivo.
2) Política parametrizada con logits -> pesos via softmax.
3) Recompensa = utilidad log(wealth_{t+1}/wealth_t).
4) Q-learning sobre estados discretizados de retornos recientes.


## 1) Imports

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(42)


## 2) Datos multiactivo sintéticos

In [ ]:

def synthetic_multi_returns(T=2000, N=4, ar=0.03, sigma=0.012, seed=42):
    rng = np.random.default_rng(seed)
    R = np.zeros((T, N))
    for i in range(N):
        for t in range(1, T):
            R[t, i] = ar * R[t-1, i] + rng.normal(0, sigma)
    return R

R = synthetic_multi_returns()
plt.figure()
plt.plot(R[:400, :])
plt.title('Rendimientos sintéticos multiactivo (primeros 400)')
plt.show()


## 3) MDP y Q-Learning con utilidad log y costos

In [ ]:

def discretize_vec(x, bins):
    return np.digitize(x, bins) - 1

class QLearnerMulti:
    def __init__(self, R, lookback=3, bins=5, gamma=0.99, alpha=0.2,
                 cost_rebalance=0.0005, temp=0.5):
        # R: T x N returns
        self.R = R
        self.T, self.N = R.shape
        self.lookback = lookback
        self.bins = bins
        self.gamma = gamma
        self.alpha = alpha
        self.cost_rebalance = cost_rebalance
        self.temp = temp
        # Discretización por cuantiles comunes
        flat = R.flatten()
        qs = np.quantile(flat, np.linspace(0,1,bins+1))[1:-1]
        self.qs = qs
        # Acciones: asignaciones discretas por argmax de logits sobre N+1 (incluye cash)
        self.A = self.N + 1  # elegir 1 activo o cash en cada t (simple)
        self.S = (bins**(self.N*lookback))  # upper bound, no se materializa completa
        # Tabla Q como dict: key=(tuple estado, accion) -> valor
        self.Q = {}

    def state_at(self, t):
        # estado: bins de los últimos 'lookback' retornos por activo
        vals = []
        for L in range(self.lookback):
            r = self.R[t-L-1]
            for i in range(self.N):
                vals.append(discretize_vec(r[i], self.qs))
        return tuple(vals)

    def boltzmann(self, q_vec):
        z = (q_vec / max(1e-8, self.temp))
        z = z - np.max(z)
        p = np.exp(z)
        return p / p.sum()

    def step(self, w, a, t):
        # a: índice 0..N => N es cash
        # construir nuevo vector de pesos one-hot
        w_new = np.zeros(self.N+1)
        w_new[a] = 1.0
        # costo de rebalanceo ~ L1
        cost = self.cost_rebalance * np.sum(np.abs(w_new - w))
        # retorno de portafolio sobre activos (cash = 0)
        r_port = np.dot(w_new[:self.N], self.R[t])
        growth = max(1e-8, 1.0 + r_port)
        reward = np.log(growth) - cost  # utilidad log
        return w_new, reward

    def train(self, epochs=3):
        # pesos iniciales en cash
        for ep in range(epochs):
            w = np.zeros(self.N+1); w[-1] = 1.0
            for t in range(self.lookback, self.T-1):
                s = self.state_at(t)
                # construir vector Q para acciones
                q_vec = np.array([self.Q.get((s, a), 0.0) for a in range(self.A)])
                p = self.boltzmann(q_vec)
                a = np.random.choice(self.A, p=p)
                w_next, r = self.step(w, a, t)
                s_next = self.state_at(t+1)
                q_next = np.array([self.Q.get((s_next, ap), 0.0) for ap in range(self.A)])
                td_target = r + self.gamma * np.max(q_next)
                td_error = td_target - self.Q.get((s, a), 0.0)
                self.Q[(s, a)] = self.Q.get((s, a), 0.0) + self.alpha * td_error
                w = w_next

    def backtest(self):
        w = np.zeros(self.N+1); w[-1] = 1.0
        wealth = [1.0]
        actions = []
        for t in range(self.lookback, self.T-1):
            s = self.state_at(t)
            q_vec = np.array([self.Q.get((s, a), 0.0) for a in range(self.A)])
            a = int(np.argmax(q_vec))
            actions.append(a)
            w, r = self.step(w, a, t)
            wealth.append(wealth[-1] * np.exp(r))  # inversa de log-utility
        return np.array(wealth), np.array(actions)

qlm = QLearnerMulti(R, lookback=3, bins=5, cost_rebalance=0.0005)
qlm.train(epochs=5)
wealth, acts = qlm.backtest()
plt.figure()
plt.plot(wealth)
plt.title('Wealth (Q-learning multi‑activo con utilidad log)')
plt.show()


## 4) Notas
- Acción simplificada a *selección de un activo*; extender a pesos continuos con política softmax paramétrica.
- Cambiar utilidad por Sharpe penalizado si se desea.
- Añadir *constraints* realistas y *transaction schedules*. 